# Lab Protocol Dashboard

This notebook provides an interactive dashboard to explore and manage both YAML protocols and Markdown protocols with YAML frontmatter.

## Features
- View all protocols in a searchable table
- Filter by protocol type (YAML or Markdown)
- Compare protocol structures
- Visualize protocol statistics

Let's start by importing the required libraries and setting up our environment.

In [ ]:
# Install dependencies if not already installed
import sys
import subprocess

def install_package(package):
    try:
        __import__(package)
        print(f"{package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully")

# Install required packages
install_package("pandas")
install_package("matplotlib")
install_package("ipywidgets")
install_package("pyyaml")
install_package("plotly")

print("\nAll dependencies are installed and ready to use.")

In [ ]:
import os
import re
import yaml
import glob
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import datetime
from IPython.display import display, HTML, Markdown

# Configure paths
WORKSPACE_ROOT = "/workspaces/docs"
PROTOCOLS_DIR = os.path.join(WORKSPACE_ROOT, "Protocols")

print(f"Workspace root: {WORKSPACE_ROOT}")
print(f"Protocols directory: {PROTOCOLS_DIR}")
print(f"Current working directory: {os.getcwd()}")

## Load Protocol Data

Now we'll load all protocol data from both YAML files and Markdown files with YAML frontmatter.

In [ ]:
def extract_frontmatter(markdown_content):
    """Extract YAML frontmatter from markdown content"""
    pattern = r"^---\n(.*?)\n---"
    match = re.search(pattern, markdown_content, re.DOTALL)
    if match:
        try:
            return yaml.safe_load(match.group(1))
        except yaml.YAMLError:
            return None
    return None

def load_protocol_files():
    """Load protocol data from both YAML and Markdown files"""
    protocols = []
    
    # Process YAML files
    yaml_files = glob.glob(os.path.join(PROTOCOLS_DIR, "*.yaml"))
    for file_path in yaml_files:
        try:
            with open(file_path, 'r') as f:
                data = yaml.safe_load(f)
                if data:
                    data['file_path'] = os.path.basename(file_path)
                    data['file_type'] = 'yaml'
                    protocols.append(data)
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    # Process Markdown files with frontmatter
    md_files = glob.glob(os.path.join(PROTOCOLS_DIR, "*.md"))
    for file_path in md_files:
        try:
            with open(file_path, 'r') as f:
                content = f.read()
                frontmatter = extract_frontmatter(content)
                if frontmatter:
                    frontmatter['file_path'] = os.path.basename(file_path)
                    frontmatter['file_type'] = 'markdown'
                    
                    # Extract content preview (first 100 chars)
                    content_without_frontmatter = re.sub(r"^---\n.*?\n---\n", "", content, flags=re.DOTALL)
                    preview = content_without_frontmatter.strip()[:100] + "..."
                    frontmatter['content_preview'] = preview
                    
                    protocols.append(frontmatter)
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    return protocols

# Load all protocols
protocols = load_protocol_files()
print(f"Loaded {len(protocols)} protocols")

# Convert to DataFrame for easier manipulation
df_protocols = pd.DataFrame(protocols)

# Fill missing values with placeholders
for col in ['id', 'name', 'version', 'description', 'author', 'created']:
    if col not in df_protocols.columns:
        df_protocols[col] = None

# Preview the dataframe
df_protocols[['file_path', 'file_type', 'id', 'name', 'version']].head()

## Protocol Dashboard

Let's create a dashboard to explore our protocols. We'll include:
1. Summary statistics
2. Interactive filtering
3. Protocol details viewer

In [ ]:
# 1. Summary statistics
yaml_count = len(df_protocols[df_protocols['file_type'] == 'yaml'])
md_count = len(df_protocols[df_protocols['file_type'] == 'markdown'])

# Create a nice HTML summary
summary_html = f"""
<div style="background-color: #f5f5f5; padding: 15px; border-radius: 10px; margin-bottom: 20px;">
    <h2 style="margin-top: 0;">Protocol Dashboard Summary</h2>
    <p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
    <p><strong>Total Protocols:</strong> {len(df_protocols)}</p>
    <ul>
        <li><strong>YAML Files:</strong> {yaml_count}</li>
        <li><strong>Markdown with Frontmatter:</strong> {md_count}</li>
    </ul>
</div>
"""

display(HTML(summary_html))

# Create a pie chart of file types
fig = px.pie(values=[yaml_count, md_count], 
             names=['YAML', 'Markdown'], 
             title='Protocol File Types',
             color_discrete_sequence=['#636EFA', '#EF553B'])
fig.update_layout(width=600, height=400)
fig.show()

## Interactive Protocol Explorer

Use the filters below to explore your protocols: